Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [6]:
NAME = "Hung-Wei Chang"
COLLABORATORS = ""

---

<a href='https://ai.meng.duke.edu'> = <img align="left" style="padding-top:10px;" src=https://storage.googleapis.com/aipi_datasets/Duke-AIPI-Logo.png>

In [7]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt

# Part 1: APIs
In this problem we are going to analyze some weather data for the month of August 2021 for the Raleigh-Durham Airport (RDU).  We will use the [U.S. NOAA Climate Data Online API](https://www.ncdc.noaa.gov/cdo-web/webservices/v2#gettingStarted) to get the data we need.  

The first thing we need to do is to get an access token to be able to make calls to the api.  You can request a free access token [here](https://www.ncdc.noaa.gov/cdo-web/token).  You will need to put your token in below to run the code, however you may delete your token if you wish before you submit your assignment (the grading script uses my own token).

The second thing we need to do is identify the station_id of the weather station located at the Raleigh-Durham Airport (referred to on the NOAA pages as 'Raleigh Airport').  You can find the id number through [this page](https://www.ncdc.noaa.gov/cdo-web/datatools/selectlocation).  You will need to search by county (Wake County) and then find the Raleigh Airport.

The third thing we need to know is the dataset id of the dataset we would like to get from the API for our station.  The dataset we will be using is 'GHCND'.

Now that we've done these, we are ready to use the API!  You can see examples of how to form the API endpoint to make calls [here](https://www.ncdc.noaa.gov/cdo-web/webservices/v2#data).  

Hint: we may need to include headers and a payload in our API call.  We should include our token within the headers, and within the payload we should include the parameters for our API call and also include 'limit'.  The purpose of 'limit' is to paginate the results so we don't receive a very large number of results back at once.  In our case we will use a limit of 500 so that we can ensure we will receive all the results we need from our single call.





In [18]:
def get_weather(token,datasetid,station_id,startdate,enddate,limit=500):
    
    if (token==None) or (station_id==None):
        raise ValueError 
     
    # YOUR CODE HERE
    url_first = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?'
    end_point = f'datasetid={datasetid}&stationid={station_id}&startdate={startdate}&enddate={enddate}&limit={limit}'
#  f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&locationid=ZIP:28801&startdate=2010-05-01&enddate=2010-05-01'

    url = url_first + end_point
    station_id = 'GHCND:USW00013722'
    response = requests.get(url = url, headers={'token':token})
    response = response.json()['results']

    return response
    

    
    raise NotImplementedError()

In [19]:
token = 'yvVNcupzVGtWZIfzsrEHKKJBCJpXjOGj'
station_id = 'GHCND:USW00013722'
datasetid = 'GHCND'
startdate = '2021-08-01'
enddate = '2021-08-31'
limit = '500'
response = get_weather(token,datasetid,station_id,startdate,enddate,limit)
print(response)

[{'date': '2021-08-01T00:00:00', 'datatype': 'AWND', 'station': 'GHCND:USW00013722', 'attributes': ',,W,', 'value': 18}, {'date': '2021-08-01T00:00:00', 'datatype': 'PRCP', 'station': 'GHCND:USW00013722', 'attributes': 'T,,W,2400', 'value': 0}, {'date': '2021-08-01T00:00:00', 'datatype': 'SNOW', 'station': 'GHCND:USW00013722', 'attributes': ',,W,', 'value': 0}, {'date': '2021-08-01T00:00:00', 'datatype': 'TAVG', 'station': 'GHCND:USW00013722', 'attributes': 'H,,S,', 'value': 276}, {'date': '2021-08-01T00:00:00', 'datatype': 'TMAX', 'station': 'GHCND:USW00013722', 'attributes': ',,W,2400', 'value': 328}, {'date': '2021-08-01T00:00:00', 'datatype': 'TMIN', 'station': 'GHCND:USW00013722', 'attributes': ',,W,2400', 'value': 239}, {'date': '2021-08-01T00:00:00', 'datatype': 'WDF2', 'station': 'GHCND:USW00013722', 'attributes': ',,W,', 'value': 350}, {'date': '2021-08-01T00:00:00', 'datatype': 'WDF5', 'station': 'GHCND:USW00013722', 'attributes': ',,W,', 'value': 30}, {'date': '2021-08-01T00

In [20]:
# Test cell
datasetid = 'GHCND'
startdate = '2021-08-01'
enddate = '2021-08-31'
limit = '500'
token = None # Replace this with your token here.  You can remove it if you like prior to submitting, I will use mine for grading
station_id = None # Replace this with station id

# Start hidden tests
# End hidden tests

response = get_weather(token,datasetid,station_id,startdate,enddate,limit)
print(response)

ValueError: 

### Question 1.1
We now need to convert our raw response data into a pandas DataFrame to perform analysis on it.  We have a number of weather parameters included in our data which you can read about [here](https://www.ncei.noaa.gov/data/global-historical-climatology-network-daily/doc/GHCND_documentation.pdf).

Complete the function `create_df()` which converts the json response data into a pandas DataFrame for analysis.  Your function should do the following:  
- Convert 'response' into a pandas DataFrame. 
- Convert the 'date' column from strings to pandas datetime format
- Filter to only the three columns we need: 'date','datatype', and 'value'  
- You will notice that 'datatype' contains the weather parameter names and 'value' contains their value each day.  Use the [pivot](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pivot.html) method to reshape your dataframe so that the columns of your dataframe are the weather parameters contained in the 'datatype' column, and the cells of the dataframe contain their values from the 'value' column.  Set the index of the reshaped dataframe to be the 'date' column
- You will notice that our temperature columns ('TAVG','TMAX','TMIN') are in units of tenths of degrees Celsius.  Let's convert these into degrees Fahrenheit (deg_F = 9/5 * deg_C + 32)

Your function should return your reshaped DataFrame (which should have the date as index and contain 15 columns representing the weather parameters).

In [21]:
def create_df(response):
    
    # YOUR CODE HERE
    df_weather = pd.DataFrame.from_dict(response, orient='columns')
    df_weather['date'] = pd.to_datetime(df_weather['date'])
    #　pd.to_datetime('13000101', format='%Y%m%d', errors='ignore')
    df_weather = df_weather[['date','datatype', 'value']]
    df_pivoted = df_weather.pivot(index="date", columns="datatype")
    df_pivoted.columns = df_pivoted.columns.droplevel()
    df_pivoted[['TAVG','TMAX','TMIN']] = df_pivoted[['TAVG','TMAX','TMIN']].apply(lambda x: (9/5 * x + 32)/10 )
    
    return df_pivoted
    raise NotImplementedError()

In [22]:
# Test cell
df = create_df(response)
display(df.head())


datatype,AWND,PRCP,SNOW,SNWD,TAVG,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5,WT01,WT02,WT03
date,,,,,,,,,,,,,,
2021-08-01,18.0,0.0,0.0,NaN,52.88,62.24,46.22,350.0,30.0,54.0,85.0,1.0,NaN,NaN
2021-08-02,32.0,0.0,0.0,0.0,49.10,56.12,40.28,50.0,50.0,72.0,89.0,NaN,NaN,NaN
2021-08-03,32.0,28.0,0.0,0.0,43.52,46.22,37.22,50.0,100.0,63.0,98.0,1.0,NaN,NaN
2021-08-04,36.0,8.0,0.0,0.0,42.26,52.16,37.22,40.0,50.0,76.0,98.0,1.0,NaN,NaN
2021-08-05,20.0,0.0,0.0,0.0,45.32,57.20,33.26,80.0,80.0,63.0,85.0,NaN,NaN,NaN


### Question 1.2
Let's now do a bit of analysis on our data by identifying which day during the month had the hottest maximum temperature.  Complete the below function `find_hottest_day()` which takes the dataframe of weather data for the month (the output of `create_df()`) as input and returns the date with the highest maximum temperature, **as a string in the exact format 'YYYY-MM-DD'**.  If there were multiple days in the month with the same max temperature, your function should return the earliest date.

In [23]:
def find_hottest_day(df):
    
    # YOUR CODE HERE
    df = df.sort_values(by=['TMAX', 'date'], ascending=[False, True])
    ts = df.index.strftime('%Y-%m-%d').get_level_values(0).values[0]
    return str(ts)
    raise NotImplementedError()

In [24]:
# Test cell
day = find_hottest_day(df)
print('Your function found the hottest day was {}'.format(day))


Your function found the hottest day was 2021-08-13


# Part 2: Web Scraping
## Exploring H1B jobs and salaries
When U.S. companies want to hire skilled foreign workers, they apply to the U.S. government for an H1B visa on behalf of the employee to be able to live and work in the country.  Certain data from all H1B visa applications, including location, job title, and salary, are publically available.  The website H1B Visa Salary Database (https://h1bdata.info/index.php) has compiled this information for all visa applications across the country from 2012-2019. We will use this dataset to perform some basic analysis of jobs obtained by H1B workers.

### Question 2.0
Look at the website and determine the api call format.  Create the url to access all jobs for all employers for the city of Raleigh for all years available.  Save the url as a string variable `data_url`

In [29]:
city = 'RALEIGH'

# YOUR CODE HERE
data_url = f'https://h1bdata.info/index.php?em=&job=&city={city}&year=All+Years'


In [30]:
# Test cell - hidden test below

### Question 2.1
Write a function `get_jobs()` that takes as input the url you just created, and uses BeautifulSoup to extract the contents of the page, find the table of jobs, and extract the header and contents of the table into a pandas dataframe called `jobs_df` which it returns from the function. The returned DataFrame `jobs_df` should contain the following columns: 'EMPLOYER','JOB TITLE','BASE SALARY','LOCATION','SUBMIT DATE','START DATE','CASE STATUS'.

In [31]:
# One way to approach writing this function might be as follows:
#   Request the page and use BeautifulSoup to extract the contents
#   Find the table of jobs on the page using its class
#   Get all rows in the table
#   Extract the header from the first row 
#   Extract the table contents from the rest of the rows into a list of lists by looping over the rows and contents of each row
#   Convert the list of lists into a pandas dataframe using the header as the column names

def get_jobs(data_url):
    
    # YOUR CODE HERE
    page = requests.get(data_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    rows = soup.find('tbody').find_all('tr')
    new_rows = rows[0].find_all('td')
    df_list = []
    for r in rows:
        new_rows = r.find_all('td')
        if new_rows[0].find('a') != None:
            final_row = [
              new_rows[0].find('a').text,
              new_rows[1].find('a').text,
              new_rows[2].text,
              new_rows[3].find('a').text,
              new_rows[4].text,
              new_rows[5].text,
              new_rows[6].text ] 
            df_list.append(final_row)
    df = pd.DataFrame( df_list, columns= [ 'EMPLOYER','JOB TITLE','BASE SALARY','LOCATION','SUBMIT DATE','START DATE','CASE STATUS']   )
    return df 
    raise NotImplementedError()
    

In [32]:
# Test cell
jobs_df = get_jobs(data_url)
display(jobs_df.head())

# Hidden tests below

,EMPLOYER,JOB TITLE,BASE SALARY,LOCATION,SUBMIT DATE,START DATE,CASE STATUS
0,ALLSCRIPTS SOFTWARE LLC,,"110,698","RALEIGH, NC",05/18/2016,09/16/2016,DENIED
1,BRIDGERA LLC,.NET ARCHITECT,"110,000","RALEIGH, NC",06/21/2016,06/21/2016,WITHDRAWN
2,BRIDGERA LLC,.NET ARCHITECT,"110,000","RALEIGH, NC",06/21/2016,06/21/2016,CERTIFIED
3,EXA DATA SOLUTIONS INC,.NET DEV PROGRAMMER,"85,426","RALEIGH, NC",03/03/2019,08/20/2019,CERTIFIED
4,REALSOFT TECHNOLOGIES LLC,.NET DEVELOPER,"60,000","RALEIGH, NC",03/11/2015,09/10/2015,CERTIFIED


### Question 2.2
We now have our data organized in a dataframe. Let's clean it up a bit by doing the following:  
- Drop any rows from the dataframe which contain null values (hint: use `dropna()`)
- Create a new column 'YEAR' which stores the year of each application as an integer  
- Convert the 'BASE SALARY' column to integer to be able to work with it (hint: before we convert to integer we must first remove the commas in the string numbers. For this we can use pandas replace(regex=True) and replace commas with '')  
- Filter the dataframe to contain only jobs where 'CASE STATUS' is "CERTIFIED"  
- Simplify our dataframe to contain only the columns EMPLOYER, JOB TITLE, BASE SALARY, LOCATION, and YEAR  

Write a function `clean_df()` which takes as input a dataframe (jobs_df) and performs the above operations.  The function should return the cleaned and filtered dataframe.

In [33]:
def clean_df(jobs_df):
    
    # YOUR CODE HERE
    df = jobs_df.replace('', np.nan)
    df = jobs_df.dropna(axis=0, how="any")
    df['YEAR'] = pd.to_datetime(df['SUBMIT DATE']).apply(lambda x: int(x.year))
    df['BASE SALARY'] = df['BASE SALARY'].str.replace(',', '', regex= True).astype(int)
    df = df.loc[ df['CASE STATUS'] == 'CERTIFIED' ]
    df = df[['EMPLOYER', 'JOB TITLE', 'BASE SALARY', 'LOCATION', 'YEAR']]
    return df
    raise NotImplementedError()
    

In [34]:
cleanjobs_df = clean_df(jobs_df)
print('There are {} unique job roles in the table \n'.format(cleanjobs_df['JOB TITLE'].nunique()))
print('Most common job titles:')
display(cleanjobs_df['JOB TITLE'].value_counts()[:10])

# Hidden tests below

There are 2606 unique job roles in the table 

Most common job titles:


SOFTWARE ENGINEER                      502
SOFTWARE DEVELOPER                     407
SENIOR SYSTEMS ANALYST JC60            317
SENIOR SOFTWARE ENGINEER               236
PROGRAMMER ANALYST                     216
POSTDOCTORAL RESEARCH SCHOLAR          184
TECHNOLOGY LEAD - US - PRACTITIONER    157
MANAGER JC50                           157
ASSISTANT PROFESSOR                    154
SYSTEMS ANALYST                        106
Name: JOB TITLE, dtype: int64

### Question 2.3
Let's see which job title has the highest median salary in a given year, **out of all jobs titles that appear at least 5 times in the year**.

Complete the below function `highest_paid_job()` which takes as input the cleaned dataframe (the output of `clean_df()`) and a year, and returns a **list of the three job titles** with the highest median salary that year, ordered with the highest median salary role first.  However, when we do this calculation we will only include job roles which appear at least 5 times within the year (because otherwise our answer may be skewed by a position for which only 1-2 candidates were hired at a very high salary).

In [35]:
def top3_highest_paid_job(df,year):
    # YOUR CODE HERE
    df_condition = df.groupby(["YEAR", 'JOB TITLE' ]).count()
    df_condition = df_condition[ df_condition['EMPLOYER'] >= 5 ]
    df_condition.reset_index(inplace=True)
    title_list = df_condition[df_condition['YEAR'] ==year]['JOB TITLE'].to_list()
    
    high_list = df[df['YEAR'] == year].groupby('JOB TITLE').median().reset_index().sort_values(by = ['BASE SALARY'], ascending = False)['JOB TITLE'].to_list()
    res = [value for value in high_list if value in title_list]

    return res[0:3]
    raise NotImplementedError()

In [36]:
# Test cell
highest_2021 = top3_highest_paid_job(cleanjobs_df,2021)
print('Highest paid roles in 2021 are: ',highest_2021)
assert highest_2021 == ['SENIOR CONSULTANT', 'SENIOR SOFTWARE ENGINEER', 'SENIOR PROJECT MANAGER - US']
print('Passed visible test')

# Hidden tests below

Highest paid roles in 2021 are:  ['SENIOR CONSULTANT', 'SENIOR SOFTWARE ENGINEER', 'SENIOR PROJECT MANAGER - US']
Passed visible test
